# Basic Training
No mixup in Training

In [1]:
import os, glob, re, cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import random

import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns

from sklearn.model_selection import train_test_split

from tensorflow import keras
from tensorflow.keras import layers, activations, optimizers, losses, metrics, initializers
from tensorflow.keras.preprocessing import image, image_dataset_from_directory
from tensorflow.keras.applications.resnet_v2 import preprocess_input, decode_predictions

import sklearn
from keras import utils
from sklearn.preprocessing import LabelEncoder

In [2]:
# dir_path points towards where to find the image dataset
dir_path = "Alzheimer's Disease"
IMAGE_SHAPE = (224, 224)

random_sample = 1

# IMPORTANT - Check these are correct

MIXUP = True # Whether to train the model with mixup
ITERS = 1 # How many mixup images to create per image
BETA = True # Whether to sample from the beta function
AUG = False # whether to augment the data

In [3]:
# Read the folders folders
directories = os.listdir(dir_path)

files = [] 
labels = []

# Read files for each directory
for folder in directories:
    
    fileList = glob.glob(f'{dir_path}/{folder}/*')
    labels.extend([folder for _ in fileList])
    files.extend(fileList)

len(files), len(labels)

(6400, 6400)

In [4]:
# Count the number of labels to get an idea of how balanced the dataset is
unique, counts = np.unique(labels, return_counts=True)
print(dict(zip(unique, counts)))

{'Demented': 3200, 'NonDem': 3200}


In [5]:
# One hot encoding

# Encode class values as integers
encoder = LabelEncoder()
encoder.fit(labels)
encoded_Y = encoder.transform(labels)

# convert integers to dummy variables (i.e. one hot encoded)
dummy_y = utils.to_categorical(encoded_Y,dtype ="uint8", num_classes = 2)

labels = np.array(dummy_y)

print(f"Classes are {encoder.classes_}")

Classes are ['Demented' 'NonDem']


In [6]:
# The data augmentation to do. This applies a random horizontal flip and a random tilt.
data_augmentation = tf.keras.Sequential([
  layers.RandomFlip("horizontal_and_vertical"),
  layers.RandomRotation((-0.05, 0.05)),
])

# create prepare_image method
# used to preprocess the image for efficientNet model

def prepare_image(file1: str, label1: str, file2: str = None, label2: str = None, beta = False, augment = True):
    """Prepares the image for the model

    Args:
        file1 (str): The path of  the first image
        label1 (str): The label of the first image
        file2 (str, optional): The path of  the second image. If not none, applies mixup. Defaults to None.
        label2 (str, optional): The label of the second image. Defaults to None.
        beta (bool, optional): If true, sample from teh beta function for mixup. Defaults to False.
        augment (bool, optional): If true, apply augmentations. Defaults to True.

    Returns:
        image, label
    """

    # Load the first image, apply augmentations
    img1 = image.load_img(file1, target_size=IMAGE_SHAPE)
    if augment:
        img1 = data_augmentation(img1)
        
    img_array = image.img_to_array(img1)
    label = label1

    if file2:

        # Load second image, apply augmentations
        img2 = image.load_img(file2, target_size=IMAGE_SHAPE)
        if augment:
            img2 = data_augmentation(img2)
        img_array2 = image.img_to_array(img2)

        # Generate alpha value for MixUp
        alpa = 0
        if beta:
            alpha = np.random.beta(0.2, 0.2, 1)
        else:
            alpha = random.random()

        # Combine the 2 images 
        img_array = (alpha * img_array) + ((1-alpha) * img_array2)
        label     = (alpha * label1) +    ((1-alpha) * label2)

    return tf.keras.applications.efficientnet.preprocess_input(img_array), label

In [7]:
# Split the data into testing, training and validation data

x_train, x_tv, y_train, y_tv = train_test_split(files, labels, test_size=.05, random_state = random_sample) 
x_test, x_val, y_test, y_val = train_test_split(x_tv, y_tv, test_size=.5, random_state = random_sample) 
x_test_A, x_test_B, y_test_A, y_test_B = train_test_split(x_test, y_test, test_size=.7, random_state = random_sample) # Have 2 test sets

In [8]:
print(len(x_test_A), len(x_test_B), len(x_val), len(x_train))

48 112 160 6080


In [9]:
# The dictionary holds list of images and for each one has its target/label
images = {
    'image_train': [], 
    'target_train': [],
    'image_test': [], 
    'target_test': [],
    'image_val': [],
    'target_val': []
}

print('Preparing the images...')

print("Preparing training data")
for file, label in zip(x_train, y_train):
    # Only go through training data, testing data should not be used for mixup


    if MIXUP == False:
        # Prepare image without mixup
        new_file, new_label = prepare_image(file, label, beta = BETA, augment = AUG)
        images['image_train'].append(new_file)
        images['target_train'].append(new_label)

    elif MIXUP == True:
        # Prepare image with mixup
        # Iterate through the number of ITERS
        for i in range(ITERS):
            diff_label = False
            
            while diff_label == False:
                # Select a random file index
                random_file_i = random.randint(0,len(files)-1)
                # Mixup the two images
                new_file, new_label = prepare_image(file, label, files[random_file_i],labels[random_file_i], beta = BETA, augment = AUG)
                
                if new_label[0] != label[0]:
                    diff_label = True

            images['image_train'].append(new_file)
            images['target_train'].append(new_label)

print("Preparing testing data")
for file, label in zip(x_test_A, y_test_A):

    # Prepare the image
    new_file, new_label = prepare_image(file, label, beta = BETA, augment = AUG)
    images['image_test'].append(new_file)
    images['target_test'].append(new_label)

print("Preparing validation data")
for file, label in zip(x_val, y_val):

    # Prepare the image
    new_file, new_label = prepare_image(file, label, beta = BETA, augment = AUG)
    images['image_val'].append(new_file)
    images['target_val'].append(new_label)

print('Finished.')

Preparing the images...
Preparing training data
Preparing testing data
Preparing validation data
Finished.


In [10]:
class_counts = [sum(col) for col in zip(*images['target_train'])]
print(f"Demented: {class_counts[0]}  Non Demented: {class_counts[1]}")

Demented: 3043.664194841916  Non Demented: 3036.335805158083


In [11]:
# convert lists to arrays 
x_train = np.array(images['image_train'])
y_train = np.array(images['target_train'])
x_test  = np.array(images['image_test'])
y_test  = np.array(images['target_test'])
x_val   = np.array(images['image_val'])
y_val   = np.array(images['target_val'])

In [12]:
# The code is creating a ResNet152V2 model with pre-trained weights from the ImageNet dataset. 
# The base model is frozen, meaning its weights will not be updated during training. 
# The code then appends additional layers on top of the base model for transfer learning.

from tensorflow.keras.applications import resnet_v2
base_model = resnet_v2.ResNet152V2(
    include_top=False,
    weights='imagenet',
    input_shape=(*IMAGE_SHAPE, 3),
    classes=3)

# Freeze the base_model
base_model.trainable = False

# append my own layers on the top of the model for Transfer Learning
x = base_model.output

# 1st conv block
x = layers.Conv2D(256, 3, padding='same')(x)
x = layers.BatchNormalization()(x)
x = layers.Activation('relu')(x)
x = layers.GlobalAveragePooling2D(keepdims = True)(x)

# 2nd conv block
x = layers.Conv2D(128, 3, padding='same')(x)
x = layers.BatchNormalization()(x)
x = layers.Activation('relu')(x)
x = layers.GlobalAveragePooling2D(keepdims = True)(x)

# 1st FC layer
x = layers.Flatten()(x) 
x = layers.Dense(64)(x)
x = layers.BatchNormalization()(x)
x = layers.Activation('relu')(x)

# 2nd FC layer
x = layers.Dense(32, activation = 'relu')(x)
x = layers.BatchNormalization()(x)
x = layers.Activation('relu')(x)
x = layers.Dropout(.2)(x)

x = layers.Dense(2, 'softmax')(x)

incept_model = keras.models.Model(inputs = base_model.input, outputs = x)

# compile the model
incept_model.compile(optimizer=optimizers.RMSprop(.001), loss = losses.categorical_crossentropy, metrics= [metrics.CategoricalAccuracy()])

# incept_model.summary()

In [13]:
EPOCHS = 30
# Trains and returns incept_model. fit ( x_train y_train )
earlyStop = keras.callbacks.EarlyStopping(patience=60) 
best_model = keras.callbacks.ModelCheckpoint(filepath='basic_model_mixup.h5', save_best_only=True) 

with tf.device('/gpu:0'):
    history = incept_model.fit(x_train, y_train, batch_size=32, epochs=EPOCHS, validation_data=(x_val, y_val), callbacks=[earlyStop, best_model])

Epoch 1/30
190/190 [==============================] - ETA: 0s - loss: 0.7143 - categorical_accuracy: 0.6010

c:\Users\Will Poulett\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


190/190 [==============================] - 446s 2s/step - loss: 0.7143 - categorical_accuracy: 0.6010 - val_loss: 1.7245 - val_categorical_accuracy: 0.5562
Epoch 2/30
190/190 [==============================] - 443s 2s/step - loss: 0.6573 - categorical_accuracy: 0.6339 - val_loss: 0.7471 - val_categorical_accuracy: 0.4062
Epoch 3/30
190/190 [==============================] - 444s 2s/step - loss: 0.6401 - categorical_accuracy: 0.6521 - val_loss: 0.7268 - val_categorical_accuracy: 0.5250
Epoch 4/30
190/190 [==============================] - 433s 2s/step - loss: 0.6225 - categorical_accuracy: 0.6618 - val_loss: 1.5072 - val_categorical_accuracy: 0.5562
Epoch 5/30
190/190 [==============================] - 459s 2s/step - loss: 0.6009 - categorical_accuracy: 0.6847 - val_loss: 1.2226 - val_categorical_accuracy: 0.4437
Epoch 6/30
190/190 [==============================] - 462s 2s/step - loss: 0.5939 - categorical_accuracy: 0.6888 - val_loss: 0.7716 - val_categorical_accuracy: 0.6375
Epoch 7/3

In [14]:
# Load the model with the best validation score
incept_model = keras.models.load_model("basic_model_mixup.h5")

incept_model.evaluate(x=x_test, y = y_test, batch_size=32, verbose=1)
#0.5833 would be wrong
incept_model.evaluate(x=x_val, y = y_val, batch_size=32, verbose=1)

5/5 [==============================] - 11s 2s/step - loss: 0.6711 - categorical_accuracy: 0.7000


[0.6711381673812866, 0.699999988079071]

In [15]:
# open train the last 100 layers
for layer in incept_model.layers[720:]:
    layer.trainable = True
    
# compile the model with new optimizer and lr=.0001
incept_model.compile(optimizer=optimizers.RMSprop(.0001), loss = losses.categorical_crossentropy, metrics=[metrics.CategoricalAccuracy()])

# incept_model.summary()

In [16]:
earlyStop = keras.callbacks.EarlyStopping(patience=60) 
best_model = keras.callbacks.ModelCheckpoint(filepath='basic_model_mixup_2.h5', save_best_only=True) 

# load the best weights
# incept_model.set_weights(best_weights)

with tf.device('/gpu:0'):
    history = incept_model.fit(x_train, y_train, batch_size=32, epochs=EPOCHS, validation_data=(x_val, y_val), callbacks=[earlyStop, best_model]) 

Epoch 1/30
190/190 [==============================] - 483s 3s/step - loss: 0.4364 - categorical_accuracy: 0.8224 - val_loss: 0.5599 - val_categorical_accuracy: 0.7188
Epoch 2/30
190/190 [==============================] - 470s 2s/step - loss: 0.4229 - categorical_accuracy: 0.8303 - val_loss: 0.5964 - val_categorical_accuracy: 0.6687
Epoch 3/30
190/190 [==============================] - 463s 2s/step - loss: 0.4164 - categorical_accuracy: 0.8349 - val_loss: 0.5153 - val_categorical_accuracy: 0.7063
Epoch 4/30
190/190 [==============================] - 466s 2s/step - loss: 0.4084 - categorical_accuracy: 0.8424 - val_loss: 0.7646 - val_categorical_accuracy: 0.7063
Epoch 5/30
190/190 [==============================] - 2351s 12s/step - loss: 0.4041 - categorical_accuracy: 0.8414 - val_loss: 0.5827 - val_categorical_accuracy: 0.7312
Epoch 6/30
190/190 [==============================] - 415s 2s/step - loss: 0.3948 - categorical_accuracy: 0.8479 - val_loss: 1.0904 - val_categorical_accuracy: 0.7

In [21]:
model = keras.models.load_model("basic_model_mixup.h5")

basic_results = model.predict(x_test)

argmax_preds = [np.argmax(r) for r in basic_results] # 0 if no tumour, 1 if yes tumour

labels = [label[1] for label in y_test] # 0 if Demented

acc = sklearn.metrics.accuracy_score(labels,argmax_preds)
f1 = sklearn.metrics.f1_score(labels,argmax_preds,pos_label=0)
print(acc,f1)

2/2 [==============================] - 5s 1s/step
0.6041666666666666 0.42424242424242425


In [18]:
# basic_results = model.predict(x_train)

# argmax_preds = [np.argmax(r) for r in basic_results] # 0 if no tumour, 1 if yes tumour

# labels = [label[1] for label in y_train] # 0 if Demented

# acc = sklearn.metrics.accuracy_score(labels,argmax_preds)
# f1 = sklearn.metrics.f1_score(labels,argmax_preds,pos_label=0)
# print(acc,f1)

In [22]:
model.evaluate(x=x_val, y = y_val, batch_size=32, verbose=1)

5/5 [==============================] - 12s 2s/step - loss: 0.6711 - categorical_accuracy: 0.7000


[0.6711381673812866, 0.699999988079071]